In [1]:
#Imports
import ROOT
import os, sys, subprocess, time
import py8settings as py8s
from py8settings import PDF
import itertools as it
import ctypes
%jsroot on

Welcome to JupyROOT 6.24/04


In [2]:
#Settings
def makePythiaSilent(pythia):
    pythia.ReadString("Init:showMultipartonInteractions = off")
    pythia.ReadString("Init:showChangedParticleData = off")
    pythia.ReadString("Init:showProcesses = off")
    pythia.ReadString("Init:showChangedSettings = off")
    pythia.ReadString("Next:numberShowInfo = 0")
    pythia.ReadString("Next:numberShowProcess = 0")
    pythia.ReadString("Next:numberShowEvent = 0")
    pythia.ReadString("Next:numberCount = 0")
    
pythia = ROOT.TPythia8(False)
makePythiaSilent(pythia)
pythia.ReadString("WeakSingleBoson:ffbar2gmZ = on")
pythia.ReadString("Random:setSeed = on")
pythia.ReadString("Random:seed = 42")
pythia.ReadString("23:onMode = off")
pythia.ReadString("23:onIfAny = 11")
pythia.ReadString("HadronLevel:Hadronize = off")
pythia.ReadString("PhaseSpace:mHatMin = 70")
pythia.ReadString("PhaseSpace:mHatMax = 110")

In [3]:
#Part 1 with FSR on
pythia.ReadString("PartonLevel:FSR = on")
particles = ROOT.TClonesArray("TParticle", 10000)
pythia.Initialize(2212 , -2212 , 630)
pythia.GenerateEvent()
#pythia.EventListing()

In [4]:
def isFinal(p,name=None):
    if name==None:
        return p.GetStatusCode()>0
    else:
        return p.GetStatusCode()>0 and p.GetName()==name

def getMinv(pair):
    p4_1=ROOT.TLorentzVector()
    p4_2=ROOT.TLorentzVector()
    pair[0].Momentum(p4_1)
    pair[1].Momentum(p4_2)
    p4=p4_1+p4_2
    return p4.M()

In [5]:
#plotting invariant mass distribution of product of decay of Z boson (e+ and e-)
nEvents=10000
h=ROOT.TH1D("h","",300,70,110)
parts=["electron", "positron"]
labels={"e-": "#e^{-}", "e+": "#e^{+}"}

def getZbosondecayCandidates(parts):
    electron = [p for p in parts if isFinal(p,"e-")]
    positron = [p for p in parts if isFinal(p,"e+")]
    return list(it.product(electron,positron))

for i in range(nEvents):
    pythia.GenerateEvent()
    pythia.ImportParticles(particles,"All")
    #pythia.EventListing()
    #for all possible e- e+ pairs we fill invariant mass histogram
    for pair in getZbosondecayCandidates(particles):
        h.Fill(getMinv(pair))
        #pythia.EventListing()
        
#normalizacja
h.Scale(1/h.GetMaximum())

 PYTHIA Warning in JunctionSplitting::SplitJunPairs: parallel junction state not allowed.  
 PYTHIA Warning in JunctionSplitting::CheckColours: Not possible to split junctions; making new colours  
 PYTHIA Error in BeamRemnants::add: failed to find physical colour state after colour reconnection  
 PYTHIA Warning in MultipartonInteractions::pTnext: weight above unity  


Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 9942003 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG
Warning in <TParticle::SetPdgCode>: PDG code 30443 unknown from TDatabasePDG


In [6]:
#Plot
c1_kstar = ROOT.TCanvas("c1","",800,600)
ROOT.gStyle.SetOptStat(0)
h.GetXaxis().SetTitle("M_{#e^{-}e^{+}} with FSR off[GeV/#it{c}^{2}]")
h.GetXaxis().SetTitleOffset(1.1)
h.GetYaxis().SetTitle("Entries")
h.SetLineColor(ROOT.kGreen)
h.SetLineWidth(2)
h.Draw("HIST")

leg=ROOT.TLegend(0.5,0.7,0.9,0.9)
leg.AddEntry(h,"all e^{-}e^{+} pairs")
leg.Draw()
c1_kstar.Draw()